In [4]:
import pickle
import json
import numpy
from glob import glob
import random

In [5]:
food_list={
    'apple': '/14_apple',
    'banana': '/03_banana',
    'carrot': '/04_carrot',
    'grape': '/18_grape',
    'cucumber': '/15_cucumber',
    'egg': '/08_egg',
    'eggplant': '/05_eggplant',
    'greenpepper': '/06_greenpepper',
    'pea': '/17_pea',
    'kiwi': '/10_kiwi',
    'lemon': '/09_lemon',
    'onion': '/13_onion',
    'orange': '/00_orange',
    'potatoes': '/12_potatoes',
    'bread': '/16_bread',
    'avocado': '/11_avocado',
    'strawberry': '/19_strawberry',
    'sweetpotato': '/02_sweetpotato',
    'tomato': '/07_tomato',
    'turnip': '/01_turnip'
    #'orange02': '/orange02'
}

# food_id contains 'foodname: pos' pairs
food_id={}
i=1
for key in food_list:
    food_id[key]=i
    i+=1

colors=['bl','br','wh']
numbers=['1','2','3']

speaker_division={}
speaker_division["train"]=[3, 5, 11, 19, 29, 30, 36, 38, 142, 144, 151]
speaker_division["val"]=[3, 5, 11, 19, 29, 30, 36, 38, 142, 144, 151]
speaker_division["test"]=[3, 5, 11, 19, 29, 30, 36, 38, 142, 144, 151]

In [7]:
def get_image_info(image_path):
    info=image_path.split("/")[8]
    name=info.split("_")[0]
    if name=="orange":
        color=info.split("_")[1]
        number=info.split("_")[2]
    else:
        color=info.split("_")[1][0:-1]
        number=info.split("_")[1][-1]
    return name, color, number

def find_audio(img_path, mode_1, mode_2='unclean',form='wav'):
    # This will return all possible captions.
    # all captions contain equal info

    food_name, color, number = get_image_info(img_path)
    # Here the basic number is a indicator for finding the correspoding wav
    # Because the orignal script (excel) is written in this way
    # Basic number + [1~6] corresponds to the 6 possible answers
    basic_number=20*(food_id[food_name]-1)+2+(eval(number)-1)*6
    audio_base_path = "/net/papilio/storage2/yhaoyuan/LAbyLM/audios_trimmed_select"
    captions = []
    if color=='wh':
        id_1=str(basic_number+1)
        id_2=str(basic_number+2)
    elif color=='bl':
        id_1=str(basic_number+3)
        id_2=str(basic_number+4)
    elif color=='br':
        id_1=str(basic_number+5)
        id_2=str(basic_number+6)
    else:
        print(color)
        print(img_path)
        raise ValueError("Not recognized color")
    if number=='1':
        name_cap=str(20*(food_id[food_name]-1)+1)
    else:
        name_cap=str(20*(food_id[food_name]-1)+2)
    if mode_1 in ["train","val","test"]:
        captions_1 = [f'/{str(speaker)}/{id_1}.wav' for speaker in speaker_division[mode_1]]
        captions_2 = [f'/{str(speaker)}/{id_2}.wav' for speaker in speaker_division[mode_1]]
        # captions_1 = [f'/{str(speaker)}/{id_1}.wav' for speaker in speaker_division[mode_1]]
        # captions_2 = [f'/{str(speaker)}/{id_2}.wav' for speaker in speaker_division[mode_1]]
    else:
        raise ValueError("Mode not recognized. Please choose from: train, valid, test.")
    return captions_1 + captions_2

def find_audio_unclean(img_path, mode_1):
    # As the data in Komatsu senpai's setting
    # Only id_4 should contain all info
    # id_1~3 should contain only partial info
    food_name, color, number = get_image_info(img_path)
    basic_number=20*(food_id[food_name]-1)+2+(eval(number)-1)*6
    audio_base_path = "/net/papilio/storage2/yhaoyuan/LAbyLM/audios_trimmed_select"
    captions = []
    name_cap = None
    name_number_color_cap = None
    there_name_number_color_cap = None
    if color=='wh':
        name_number_color_cap=str(basic_number+1)
        there_name_number_color_cap=str(basic_number+2)
    elif color=='bl':
        name_number_color_cap=str(basic_number+3)
        there_name_number_color_cap=str(basic_number+4)
    elif color=='br':
        name_number_color_cap=str(basic_number+5)
        there_name_number_color_cap=str(basic_number+6)
    else:
        print(color)
        print(img_path)
        raise ValueError("Not recognized color")
    if number=='1':
        name_cap=str(20*(food_id[food_name]-1)+1)
    else:
        name_cap=str(20*(food_id[food_name]-1)+2)
    if mode_1 in ["train","val","test"]:
        speaker_list = random.sample(speaker_division["train"], 2) # sample $number of speakers
        captions_name = [f'/{str(speaker)}/{name_cap}.wav' for speaker in speaker_list]
        captions_all = [f'/{str(speaker)}/{name_number_color_cap}.wav' for speaker in speaker_list]\
            + [f'/{str(speaker)}/{there_name_number_color_cap}.wav' for speaker in speaker_list]
        # caption_1=audio_source_path+'/'+str(choice(speaker_division[mode_1]))+'/'+id_1+'.'+form
        # caption_2=audio_source_path+'/'+str(choice(speaker_division[mode_1]))+'/'+id_2+'.'+form
        # caption_3=audio_source_path+'/'+str(choice(speaker_division[mode_1]))+'/'+id_3+'.'+form
    else:
        raise ValueError("Mode not recognized. Please choose from: train, valid, test.")
    return captions_name + random.sample(captions_all, 1)

In [8]:
img_paths = "../../data/food_image_split.json"
with open(img_paths, "r") as f:
    img_paths = json.load(f)

audio_base_path = "/net/papilio/storage2/yhaoyuan/LAbyLM/audios_trimmed_select"
image_base_path = "/net/tateha/storage2/database/spolacq/FoodImagesA"

In [9]:
food_dataset = {}
food_dataset["audio_base_path"] = audio_base_path
food_dataset["image_base_path"] = image_base_path
food_dataset["data"] = {}

In [10]:
for mode in ["train","val","test"]:
    data_list = []
    for img in img_paths[mode]:
        data = {}
        data["image"] = img[len(image_base_path):]
        data["audio"] = find_audio_unclean(img, mode)
        data_list.append(data)
        print(data)
    food_dataset["data"][mode] = data_list

{'image': '/14_apple/apple_wh1_24.jpg', 'audio': ['/11/1.wav', '/3/1.wav', '/11/4.wav']}
{'image': '/14_apple/apple_wh1_18.jpg', 'audio': ['/11/1.wav', '/19/1.wav', '/19/3.wav']}
{'image': '/14_apple/apple_bl3_12.jpg', 'audio': ['/151/2.wav', '/38/2.wav', '/151/17.wav']}
{'image': '/14_apple/apple_br2_08.jpg', 'audio': ['/5/2.wav', '/29/2.wav', '/5/14.wav']}
{'image': '/14_apple/apple_bl1_17.jpg', 'audio': ['/19/1.wav', '/29/1.wav', '/19/6.wav']}
{'image': '/14_apple/apple_bl1_03.jpg', 'audio': ['/29/1.wav', '/30/1.wav', '/30/5.wav']}
{'image': '/14_apple/apple_wh3_08.jpg', 'audio': ['/11/2.wav', '/36/2.wav', '/11/16.wav']}
{'image': '/14_apple/apple_bl1_02.jpg', 'audio': ['/19/1.wav', '/151/1.wav', '/19/6.wav']}
{'image': '/14_apple/apple_bl3_07.jpg', 'audio': ['/29/2.wav', '/3/2.wav', '/3/18.wav']}
{'image': '/14_apple/apple_bl3_13.jpg', 'audio': ['/29/2.wav', '/11/2.wav', '/29/18.wav']}
{'image': '/14_apple/apple_wh1_19.jpg', 'audio': ['/144/1.wav', '/3/1.wav', '/3/4.wav']}
{'image'

In [11]:
with open("food_dataset_sentence_embedding_3.json", "w") as f:
    json.dump(food_dataset, f)

In [12]:
# check the split
import os
for data in food_dataset["data"]["test"]:
    assert os.path.isfile(food_dataset["image_base_path"]+data["image"]), "Not a valid audio file"
    assert os.path.isfile(food_dataset["audio_base_path"]+data["audio"][0]), "Not a valid audio file"